### 1) Cite 5 diferenças entre o Random Forest e o AdaBoost

|Random Forest| AdaBoost|
|-------------|---------|
|Floresta de árvores| Floresta de _"Stumps"_|
|Árvores independentes|Árvore influenciando na próxima|
|Todas as árvores têm o mesmo peso| Respostas têm pesos diferentes|
|Baseado na técnica _"bagging"_| Baseado na técnica _"boosting"_|
|Grande chance de ocorrer _overfit_|Quase nunca ocorre  _overfit_ |

### 2) Exemplo Scikit-learn AdaBoost

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib notebook

X, y = load_iris(return_X_y=True)
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()


0.9466666666666665

### 3) Cite 5 Hyperparametros importantes no AdaBoost
- n_estimators: Número máximo de árvores no qual o _boost_ é encerrado;
- base_estimator: O estimador base a partir do qual o conjunto _boost_ é construído.
- learning_rate: Peso aplicado aos classificadores em cada iteração;
- algorithm: Define qual algoritmo será utilizado no cálculo, podendo ser _SAMME_ ou _SAMME.R_;
- random_state: Controla a semente de randomização em cada iteração.

### 4) Utilizando do GridSearch para encontrar os melhores hiperparametros

In [2]:
def calcula_gini(RESP, PD):
    #AUC
    auc = roc_auc_score(RESP, PD,multi_class='ovr')
           
    #Gini
    gini = 2*auc -1
    return gini

def executar_validador(X,y):
    validador = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
    for treino_id, teste_id in validador.split(X,y):
        X_train, X_test = X[treino_id], X[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]
    return X_train, X_test,y_train,y_test

In [3]:
X_train, X_test,y_train,y_test = executar_validador(X,y)

In [4]:
%%time
lista_gini_test=[]
lista_gini_train=[]

grid = list(range(100, 1001, 150))

for num_arvores in grid:
    clf = AdaBoostClassifier(n_estimators=num_arvores)
    
    clf.fit(X_train,y_train.ravel())

    gini_test  = calcula_gini(y_test, clf.predict_proba(X_test))
    gini_train = calcula_gini(y_train, clf.predict_proba(X_train))

    lista_gini_test.append(gini_test)
    lista_gini_train.append(gini_train)    

Wall time: 4.4 s


In [9]:
zip_gini_train = zip(list(range(100, 1001, 150)), lista_gini_train)
zip_gini_test = zip(list(range(100, 1001, 150)), lista_gini_test)

dict_gini_train = dict(zip_gini_train)
dict_gini_test = dict(zip_gini_test)


series_gini_train = pd.Series(dict_gini_train)
series_gini_test = pd.Series(dict_gini_test)


pd.DataFrame(dict_gini_train,dict_gini_test)

,100,250,400,550,700,850,1000
100,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
250,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
400,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
550,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
700,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
850,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
1000,0.994558,0.994014,0.994558,0.994014,0.994558,0.994014,0.994558
